In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("landsat").getOrCreate()

24/06/24 20:01:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
import cv2
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType
from pyspark.sql import Row

def read_image(image_path):
    img = cv2.imread(image_path)
    _, buffer = cv2.imencode('.jpg', img)
    return buffer.tobytes()


read_image_udf = udf(lambda z: read_image(z), BinaryType())

dubaj = ["dubaj_2013_1.jpg", "dubaj_2013_2.jpg", "dubaj_2013_3.jpg", "dubaj_2013_4.jpg", "dubaj_2024_1.jpg", "dubaj_2024_2.jpg", "dubaj_2024_3.jpg", "dubaj_2024_4.jpg"]
las = ["las_2013_1.jpg", "las_2013_2.jpg", "las_2013_3.jpg", "las_2013_4.jpg", "las_2024_1.jpg", "las_2024_2.jpg", "las_2024_3.jpg", "las_2024_4.jpg"]
qatar = ["qatar_2013_1.jpg", "qatar_2013_2.jpg", "qatar_2013_3.jpg", "qatar_2013_4.jpg", "qatar_2024_1.jpg", "qatar_2024_2.jpg", "qatar_2024_3.jpg", "qatar_2024_4.jpg"]
qatar2 = ["qatar2_2013_1.jpg", "qatar2_2013_2.jpg", "qatar2_2013_3.jpg", "qatar2_2013_4.jpg", "qatar2_2024_1.jpg", "qatar2_2024_2.jpg", "qatar2_2024_3.jpg", "qatar2_2024_4.jpg"]
utah = ["utah_2013_1.jpg", "utah_2013_2.jpg", "utah_2013_3.jpg", "utah_2013_4.jpg", "utah_2024_1.jpg", "utah_2024_2.jpg", "utah_2024_3.jpg", "utah_2024_4.jpg"]

data_path = dubaj+las+qatar+qatar2+utah

image_df = spark.createDataFrame([Row(path=path) for path in data_path])

image_df = image_df.withColumn("image", read_image_udf("path"))



In [12]:
print(image_df.show(100))

+-----------------+--------------------+
|             path|               image|
+-----------------+--------------------+
| dubaj_2013_1.jpg|[FF D8 FF E0 00 1...|
| dubaj_2013_2.jpg|[FF D8 FF E0 00 1...|
| dubaj_2013_3.jpg|[FF D8 FF E0 00 1...|
| dubaj_2013_4.jpg|[FF D8 FF E0 00 1...|
| dubaj_2024_1.jpg|[FF D8 FF E0 00 1...|
| dubaj_2024_2.jpg|[FF D8 FF E0 00 1...|
| dubaj_2024_3.jpg|[FF D8 FF E0 00 1...|
| dubaj_2024_4.jpg|[FF D8 FF E0 00 1...|
|   las_2013_1.jpg|[FF D8 FF E0 00 1...|
|   las_2013_2.jpg|[FF D8 FF E0 00 1...|
|   las_2013_3.jpg|[FF D8 FF E0 00 1...|
|   las_2013_4.jpg|[FF D8 FF E0 00 1...|
|   las_2024_1.jpg|[FF D8 FF E0 00 1...|
|   las_2024_2.jpg|[FF D8 FF E0 00 1...|
|   las_2024_3.jpg|[FF D8 FF E0 00 1...|
|   las_2024_4.jpg|[FF D8 FF E0 00 1...|
| qatar_2013_1.jpg|[FF D8 FF E0 00 1...|
| qatar_2013_2.jpg|[FF D8 FF E0 00 1...|
| qatar_2013_3.jpg|[FF D8 FF E0 00 1...|
| qatar_2013_4.jpg|[FF D8 FF E0 00 1...|
| qatar_2024_1.jpg|[FF D8 FF E0 00 1...|
| qatar_2024_2.j